<a href="https://colab.research.google.com/github/divyanilharsh/C-Plus-Plus/blob/master/pcr_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install nsetools yfinance



In [ ]:
import requests
import csv
import time
import yfinance as yf
from datetime import datetime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def fetch_nifty_current_price():
    """Fetches the current NIFTY price using yfinance."""
    ticker = "^NSEI"
    try:
        nifty = yf.Ticker(ticker)
        data = nifty.history(period="1d")
        current_price = data['Close'].iloc[-1]
        return current_price
    except Exception as e:
        print(f"Error fetching NIFTY price: {e}")
        return None

def fetch_option_chain_data():
    """Fetches the option chain data from NSE using web scraping."""
    url = "https://www.nseindia.com/option-chain"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.nseindia.com',
    }

    # Use session to manage cookies and headers
    session = requests.Session()
    session.headers.update(headers)

    try:
        # Initial request to get cookies
        response = session.get(url)
        cookies = response.cookies.get_dict()

        # Fetch the option chain data
        option_chain_url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
        response = session.get(option_chain_url, headers=headers, cookies=cookies)
        response.raise_for_status()
        data = response.json()
        return data['records']['data']
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
    except ValueError:
        print("Response content is not valid JSON.")
    print("Failed to fetch option chain data after multiple attempts.")
    return None

def filter_strike_prices(option_chain_data, current_price, x, expiry_date):
    """Filters the option chain data to include x strike prices above and below the current NIFTY price."""
    strike_prices = sorted(set([data['strikePrice'] for data in option_chain_data]))
    filtered_strikes = [price for price in strike_prices if current_price - x*50 <= price <= current_price + x*50]
    filtered_data = [data for data in option_chain_data if data['strikePrice'] in filtered_strikes and data['expiryDate'] == expiry_date]
    return filtered_data

def calculate_pcr(option_chain_data):
    """Calculates the PCR (Put-Call Ratio) from the option chain data."""
    total_pe_open_interest = sum(float(data['PE']['openInterest']) for data in option_chain_data if 'PE' in data)
    total_ce_open_interest = sum(float(data['CE']['openInterest']) for data in option_chain_data if 'CE' in data)

    pcr = total_pe_open_interest / total_ce_open_interest if total_ce_open_interest != 0 else 0
    return pcr, total_ce_open_interest, total_pe_open_interest

def append_to_csv(file_path, pcr, timestamp, total_ce_open_interest, total_pe_open_interest):
    """Appends the PCR ratio and timestamp to a CSV file."""
    try:
        with open(file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['PCR', 'Timestamp', 'Total CE Open Interest', 'Total PE Open Interest'])  # Write header only if file is empty
            writer.writerow([pcr, timestamp, total_ce_open_interest, total_pe_open_interest])
        print(f"Data has been appended to {file_path}")
    except IOError:
        print("I/O error")

def main():
    """Main function to fetch, filter, calculate ratios, and write option chain data to a CSV file periodically."""
    # Fetch current NIFTY price
    current_price = fetch_nifty_current_price()
    if current_price is None:
        print("Failed to fetch current NIFTY price. Exiting...")
        return

    print(f"Current NIFTY Price: {current_price}")

    # Fetch option chain data from NSE
    option_chain_data = fetch_option_chain_data()
    if option_chain_data is None:
        print("Failed to fetch option chain data. Exiting...")
        return

    # Get all expiry dates
    expiry_dates = sorted(set(data['expiryDate'] for data in option_chain_data))
    print(f"Expiry Dates: {expiry_dates}")

    # Get user input for the expiry date and number of strike prices
    expiry_date_input = input(f"Please enter your desired expiry date from the list above (YYYY-MM-DD): ")
    if expiry_date_input not in expiry_dates:
        print("Invalid expiry date. Please enter a valid date from the list.")
        return

    x = int(input("Enter the number of strike prices above and below the current price to include: "))

    # Define the output CSV file path on Google Drive
    output_file_path = '/content/drive/My Drive/realtime_pcr_data.csv'

    while True:
        # Fetch option chain data from NSE again
        option_chain_data = fetch_option_chain_data()
        if option_chain_data is None:
            print("Failed to fetch option chain data. Exiting...")
            return

        # Filter the data
        filtered_data = filter_strike_prices(option_chain_data, current_price, x, expiry_date_input)

        # Calculate PCR ratio
        pcr, total_ce_open_interest, total_pe_open_interest = calculate_pcr(filtered_data)

        # Get current timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Append the data to the CSV file
        append_to_csv(output_file_path, pcr, timestamp, total_ce_open_interest, total_pe_open_interest)

        # Wait for a specified interval before fetching the data again (e.g., 30 seconds)
        time.sleep(30)

if __name__ == "__main__":
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current NIFTY Price: 22530.69921875
Expiry Dates: ['04-Jul-2024', '06-Jun-2024', '13-Jun-2024', '20-Jun-2024', '24-Dec-2025', '24-Jun-2027', '25-Jul-2024', '25-Jun-2026', '26-Dec-2024', '26-Jun-2025', '26-Sep-2024', '27-Jun-2024', '27-Mar-2025', '28-Dec-2028', '29-Aug-2024', '29-Jun-2028', '30-Dec-2027', '31-Dec-2026']
Please enter your desired expiry date from the list above (YYYY-MM-DD): 04-Jul-2024
Enter the number of strike prices above and below the current price to include: 10
Data has been appended to /content/drive/My Drive/realtime_pcr_data.csv
Data has been appended to /content/drive/My Drive/realtime_pcr_data.csv
Data has been appended to /content/drive/My Drive/realtime_pcr_data.csv
Data has been appended to /content/drive/My Drive/realtime_pcr_data.csv
Data has been appended to /content/drive/My Drive/realtime_pcr_data.csv
Data has been appended 